# persistent in langgraph
Persistence in LangGraph means saving the memory and state of your AI workflow (graph) so it doesn’t get lost when something goes wrong or when you want to resume later.

Think of it like a save game feature in a video game.

- Without persistence → if the game crashes, you lose progress.

- With persistence → you reload from the last checkpoint.

So in LangGraph, persistence allows your workflow to survive crashes, restarts, or interruptions.

# Fault tolerance
Fault tolerance means the system can recover from errors without starting everything from scratch.

Persistence helps by:

- Saving progress at certain steps (checkpoints).

- If an error or crash happens, the graph reloads from the last checkpoint instead of starting at the beginning.

- This makes the system more reliable and faster because you don’t redo completed steps.

🔹 Example:
Imagine you are building a workflow that processes a customer’s request in 5 steps:

- Validate user input

- Retrieve information from a database

- Summarize the data

- Generate a response

- Send it back to the user

If the system crashes at step 4:

- Without persistence → you must start again from step 1.

- With persistence → you restart from step 3 or 4 (where you last saved).


# checkpointers in persistent
Checkpointers are like “save points” in a workflow.

- They store the state (data, inputs, outputs) of the workflow at specific moments.

- Later, you can reload the workflow from that checkpoint.

🔹 Example:
Think of an online exam system:

- Every 5 minutes, it autosaves your answers (checkpoints).

- If your laptop shuts down, when you log back in, your answers up to the last save are still there.

In LangGraph, checkpointers do the same thing for AI workflows.

# Threads in persistent

A thread is like a separate conversation or workflow instance that gets its own saved state.

- Each thread has its own memory and checkpoints.

- You can pause one thread and start another, then come back later.

🔹 Example:
Think of WhatsApp chats:

- Each chat with a person is like a “thread.”

- Messages are saved (persistent), so when you reopen the chat, you see the history.

- Each conversation is independent of the others.

In LangGraph, threads let you manage multiple workflows (conversations, tasks, or jobs) at once.

# benefits of Persistent
- Short term Memory
- Fault Tolerance
- Human in the loop
- Time Travel

# MemorySaver vs InMemorySaver
- InMemorySaver → Very simple saver, keeps checkpoints only in RAM, lost when program ends. Good for quick testing.

- MemorySaver → Higher-level saver that organizes checkpoints per thread for LangGraph workflows. Still in-memory, but structured for persistence during execution.

In [41]:
from langgraph.graph import StateGraph,START,END
from langchain_groq import ChatGroq
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import BaseMessage,HumanMessage
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from typing import  Annotated,TypedDict
load_dotenv()

True

In [42]:
class state(TypedDict):
    topic:str
    joke:str
    explain:str


In [43]:
llm=ChatGroq(model='llama-3.1-8b-instant')

In [44]:
def joke_gen(state:state):
    prompt=f"Generate the short joke on the foolowing topic :{state['topic']}"
    response=llm.invoke(prompt)
    return {'joke':response}

In [45]:
def joke_exp(state:state):
    prompt=f"explain the following joke in urdu {state['topic']}"
    exp=llm.invoke(prompt)
    return {'explain':exp}

In [46]:
graph=StateGraph(state)
graph.add_node('joke_gen',joke_gen)
graph.add_node('joke_exp',joke_exp)

graph.add_edge(START,'joke_gen')
graph.add_edge('joke_gen','joke_exp')
graph.add_edge('joke_exp',END)

checkpointer=InMemorySaver()
chatbot=graph.compile(checkpointer=checkpointer)

In [47]:
thread_id='1'

In [48]:
config={'configurable':{'thread_id':thread_id}}
result=chatbot.invoke(
    {'topic':'pizza'},
    config=config
)

In [49]:
result

{'topic': 'pizza',
 'joke': AIMessage(content='Why was the pizza in a bad mood? \n\nBecause it was feeling a little crusty.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 46, 'total_tokens': 66, 'completion_time': 0.016608481, 'prompt_time': 0.005815748, 'queue_time': 0.090975112, 'total_time': 0.022424229}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_c40956ddc4', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--9a162ff8-f8c6-432b-a42e-74563bee7c01-0', usage_metadata={'input_tokens': 46, 'output_tokens': 20, 'total_tokens': 66}),
 'explain': AIMessage(content='یہ جملہ ایک جوک ہے جس میں ایک شخص نے ایک پیتزا کے بارے میں کہا ہے۔\n\n"پیتزا"\n\nاس جملے کا مطلب ہے "پیتزا" ہی جس کا حوالہ دیتا ہے۔ یہ ایک ایسی بات ہے جو صرف "پیتزا" کا ذکر کرتی ہے، جیسے کہ ایک شخص کے بھاگ جانے کی وجہ "پیتزا" ہو۔\n\nیہ جوک اس بات پر مبنی ہے کہ پیتزا ایک ایسا کھانا ہے جس کا ذکر کرنا بہت آسان ہے، لیکن جب کوئی 

# Fault tolerance

In [50]:
def joke_gen1(state:state):
    prompt=f"Generate the short joke on the foolowing topic :{state['topic']}"
    response=llm.invoke(prompt)
    print('Generation completed...')
    return {'joke':response}

In [51]:
import time # let llm take time to resposne so we can interrupt our  system for trying fault tolerance
def joke_exp1(state:state):
    prompt=f"explain the following joke in urdu {state['topic']}"
    time.sleep(10)
    exp=llm.invoke(prompt)
    print('Explanation completed...')

    return {'explain':exp}

In [52]:
graph=StateGraph(state)
graph.add_node('joke_gen1',joke_gen1)
graph.add_node('joke_exp1',joke_exp1)

graph.add_edge(START,'joke_gen1')
graph.add_edge('joke_gen1','joke_exp1')
graph.add_edge('joke_exp1',END)

checkpointer=InMemorySaver()
chatbot=graph.compile(checkpointer=checkpointer)

In [53]:
thread_id_n='2'

In [54]:
config2={'configurable':{'thread_id':thread_id_n}}
result=chatbot.invoke(
    {'topic':'pizza'},
    config=config2
)

Generation completed...
Explanation completed...


In [55]:
result

{'topic': 'pizza',
 'joke': AIMessage(content='Why was the pizza in a bad mood? \nBecause it was feeling crusty.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 46, 'total_tokens': 64, 'completion_time': 0.015446919, 'prompt_time': 0.059389952, 'queue_time': 0.158662008, 'total_time': 0.074836871}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_2115512ff6', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--7614331d-2b4c-4849-969c-c9688a115c78-0', usage_metadata={'input_tokens': 46, 'output_tokens': 18, 'total_tokens': 64}),
 'explain': AIMessage(content='یہ جوک یہ ہے:\n\nپیزا\n\nان کی وضاحت یہ ہے:\n\nایک شخص نے اپنے دوست کو پہنچایا اور کہا، "آج تو میرے لیے ہی پیزا لائی ہوں، لیکن میں نے سوچا ہے کہ تو بھی پیزا کھانا چاہے ہو گا۔"\n\nاس کے دوست نے پوچھا، "کیا تو ہے؟ میں نے تو پہلے ہی کہا تھا کہ مجھے پیزا نہیں کھانے دے۔"\n\nپہلے شخص نے کہا، "ایسا ہی تو نہیں، تو نے کہا تھا میں بھی پیزا کھانا

In [56]:
config2={'configurable':{'thread_id':thread_id_n}}
result=chatbot.invoke(
    None,
    config=config2
)

In [57]:
list(chatbot.get_state_history(config=config2))

[StateSnapshot(values={'topic': 'pizza', 'joke': AIMessage(content='Why was the pizza in a bad mood? \nBecause it was feeling crusty.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 46, 'total_tokens': 64, 'completion_time': 0.015446919, 'prompt_time': 0.059389952, 'queue_time': 0.158662008, 'total_time': 0.074836871}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_2115512ff6', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--7614331d-2b4c-4849-969c-c9688a115c78-0', usage_metadata={'input_tokens': 46, 'output_tokens': 18, 'total_tokens': 64}), 'explain': AIMessage(content='یہ جوک یہ ہے:\n\nپیزا\n\nان کی وضاحت یہ ہے:\n\nایک شخص نے اپنے دوست کو پہنچایا اور کہا، "آج تو میرے لیے ہی پیزا لائی ہوں، لیکن میں نے سوچا ہے کہ تو بھی پیزا کھانا چاہے ہو گا۔"\n\nاس کے دوست نے پوچھا، "کیا تو ہے؟ میں نے تو پہلے ہی کہا تھا کہ مجھے پیزا نہیں کھانے دے۔"\n\nپہلے شخص نے کہا، "ایسا ہی تو نہیں، تو نے کہا تھ

# Time Travel

In [58]:
config2={'configurable':{'thread_id':thread_id_n}}
result=chatbot.invoke(
    {'topic':'burger'},
    config=config2
)

Generation completed...
Explanation completed...


In [59]:
list(chatbot.get_state_history(config=config2))

[StateSnapshot(values={'topic': 'burger', 'joke': AIMessage(content='Why was the burger in a bad mood? \n\nBecause it had a beef with everything.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 46, 'total_tokens': 65, 'completion_time': 0.027816835, 'prompt_time': 0.093438171, 'queue_time': 0.249228309, 'total_time': 0.121255006}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_c40956ddc4', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--059d69f8-5cf2-4344-a35a-ab8b79d87772-0', usage_metadata={'input_tokens': 46, 'output_tokens': 19, 'total_tokens': 65}), 'explain': AIMessage(content='Kuchh zameen se zyada nahin hai, lekin burger nahin bhi hai. (Kuchh zameen se zyada nahin hai) ishara hai kisi cheez ki, aur (burger nahin bhi hai) ishara hai kisi cheez ki jise burger kehte hain, lekin yeh kuchh zameen se zyada nahin hai. \n\nIs joke ki maanani, yeh ek burger hai jo kisi zameen se adhik

# persistent in langgraph
Persistence in LangGraph means saving the memory and state of your AI workflow (graph) so it doesn’t get lost when something goes wrong or when you want to resume later.

Think of it like a save game feature in a video game.

- Without persistence → if the game crashes, you lose progress.

- With persistence → you reload from the last checkpoint.

So in LangGraph, persistence allows your workflow to survive crashes, restarts, or interruptions.

# Fault tolerance
Fault tolerance means the system can recover from errors without starting everything from scratch.

Persistence helps by:

- Saving progress at certain steps (checkpoints).

- If an error or crash happens, the graph reloads from the last checkpoint instead of starting at the beginning.

- This makes the system more reliable and faster because you don’t redo completed steps.

🔹 Example:
Imagine you are building a workflow that processes a customer’s request in 5 steps:

- Validate user input

- Retrieve information from a database

- Summarize the data

- Generate a response

- Send it back to the user

If the system crashes at step 4:

- Without persistence → you must start again from step 1.

- With persistence → you restart from step 3 or 4 (where you last saved).


# checkpointers in persistent
Checkpointers are like “save points” in a workflow.

- They store the state (data, inputs, outputs) of the workflow at specific moments.

- Later, you can reload the workflow from that checkpoint.

🔹 Example:
Think of an online exam system:

- Every 5 minutes, it autosaves your answers (checkpoints).

- If your laptop shuts down, when you log back in, your answers up to the last save are still there.

In LangGraph, checkpointers do the same thing for AI workflows.

# Threads in persistent

A thread is like a separate conversation or workflow instance that gets its own saved state.

- Each thread has its own memory and checkpoints.

- You can pause one thread and start another, then come back later.

🔹 Example:
Think of WhatsApp chats:

- Each chat with a person is like a “thread.”

- Messages are saved (persistent), so when you reopen the chat, you see the history.

- Each conversation is independent of the others.

In LangGraph, threads let you manage multiple workflows (conversations, tasks, or jobs) at once.

# benefits of Persistent
- Short term Memory
- Fault Tolerance
- Human in the loop
- Time Travel

# MemorySaver vs InMemorySaver
- InMemorySaver → Very simple saver, keeps checkpoints only in RAM, lost when program ends. Good for quick testing.

- MemorySaver → Higher-level saver that organizes checkpoints per thread for LangGraph workflows. Still in-memory, but structured for persistence during execution.

In [62]:
from langgraph.graph import StateGraph,START,END
from langchain_groq import ChatGroq
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import BaseMessage,HumanMessage
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from typing import  Annotated,TypedDict
load_dotenv()

True

In [63]:
class state(TypedDict):
    topic:str
    joke:str
    explain:str


In [64]:
llm=ChatGroq(model='llama-3.1-8b-instant')

In [65]:
def joke_gen(state:state):
    prompt=f"Generate the short joke on the foolowing topic :{state['topic']}"
    response=llm.invoke(prompt)
    return {'joke':response}

In [66]:
def joke_exp(state:state):
    prompt=f"explain the following joke in urdu {state['topic']}"
    exp=llm.invoke(prompt)
    return {'explain':exp}

In [67]:
graph=StateGraph(state)
graph.add_node('joke_gen',joke_gen)
graph.add_node('joke_exp',joke_exp)

graph.add_edge(START,'joke_gen')
graph.add_edge('joke_gen','joke_exp')
graph.add_edge('joke_exp',END)

checkpointer=InMemorySaver()
chatbot=graph.compile(checkpointer=checkpointer)

In [68]:
thread_id='1'

In [69]:
config={'configurable':{'thread_id':thread_id}}
result=chatbot.invoke(
    {'topic':'pizza'},
    config=config
)

In [70]:
result

{'topic': 'pizza',
 'joke': AIMessage(content='Why was the pizza in a bad mood? \n\nBecause it was feeling crusty.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 46, 'total_tokens': 64, 'completion_time': 0.016073858, 'prompt_time': 0.015830292, 'queue_time': 0.089567698, 'total_time': 0.03190415}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_7083106d2c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--43176fa6-b304-4ef2-b70f-4ca627fbcf1e-0', usage_metadata={'input_tokens': 46, 'output_tokens': 18, 'total_tokens': 64}),
 'explain': AIMessage(content='Pizza کا یہ جوک ہے:\n\n" Pizza ایک خاتون سے پوچھا گیا کہ وہ کیا ہے۔ وہ کہنے لگی کہ میں ایک پیزا ہوں۔ پیزا نے کہا، \'کیا آپ کے ماں باپ بھی پیزا ہیں؟\' خاتون نے کہا، \'ہاں، وہ بھی پیزا ہیں۔\' پیزا نے کہا، \'تو آپ کے ڈیڈی کا داڑھی کہاں ہے؟\' خاتون نے کہا، \'وہ میرے سر کے اوپر ہیں۔\' پیزا نے کہا، \'تو آپ کے باپ کا داڑھی کہاں ہے؟\' خاتون ن

# Fault tolerance

In [ ]:
def joke_gen1(state:state):
    prompt=f"Generate the short joke on the foolowing topic :{state['topic']}"
    response=llm.invoke(prompt)
    print('Generation completed...')
    return {'joke':response}

In [ ]:
import time # let llm take time to resposne so we can interrupt our  system for trying fault tolerance
def joke_exp1(state:state):
    prompt=f"explain the following joke in urdu {state['topic']}"
    time.sleep(10)
    exp=llm.invoke(prompt)
    print('Explanation completed...')

    return {'explain':exp}

In [ ]:
graph=StateGraph(state)
graph.add_node('joke_gen1',joke_gen1)
graph.add_node('joke_exp1',joke_exp1)

graph.add_edge(START,'joke_gen1')
graph.add_edge('joke_gen1','joke_exp1')
graph.add_edge('joke_exp1',END)

checkpointer=InMemorySaver()
chatbot=graph.compile(checkpointer=checkpointer)

In [ ]:
thread_id_n='2'

In [ ]:
config2={'configurable':{'thread_id':thread_id_n}}
result=chatbot.invoke(
    {'topic':'pizza'},
    config=config2
)

Generation completed...


KeyboardInterrupt: 

In [ ]:
result

{'topic': 'pizza',
 'joke': AIMessage(content='Why was the pizza in a bad mood? \n\nBecause it was feeling a little crusty.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 46, 'total_tokens': 66, 'completion_time': 0.016743983, 'prompt_time': 0.005746572, 'queue_time': 0.048494358, 'total_time': 0.022490555}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_c40956ddc4', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--f941491b-1c03-48ef-9578-2cf1f92f58b3-0', usage_metadata={'input_tokens': 46, 'output_tokens': 20, 'total_tokens': 66}),
 'explain': AIMessage(content='Koi joke hai: Pizza\n\nYeh joke yeh hai: koi pizza parlor hai jahan par ek customer aata hai aur order karta hai: "Maine apne dost ko pizza khaane ke liye kaha hai, lekin wo pizza kheena nahin khata. Main apne dost ko pizza khaane ke liye bolun, lekin use pizza kheena nahin khana hai."\n\nPizza parlor owner bolta hai: "Aapko

In [ ]:
config2={'configurable':{'thread_id':thread_id_n}}
result=chatbot.invoke(
    None,
    config=config2
)

Explanation completed...


In [ ]:
list(chatbot.get_state_history(config=config2))

[StateSnapshot(values={'topic': 'pizza', 'joke': AIMessage(content='Why was the pizza in a bad mood? \n\nBecause it was feeling a little crusty.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 46, 'total_tokens': 66, 'completion_time': 0.016620453, 'prompt_time': 0.005892534, 'queue_time': 0.049852006, 'total_time': 0.022512987}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_7083106d2c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--13a56d55-6fb0-472e-b74e-5109d99afa62-0', usage_metadata={'input_tokens': 46, 'output_tokens': 20, 'total_tokens': 66}), 'explain': AIMessage(content='Kuchh logon ko yeh joke pasand aati hai:\n\nPizza ki zuban ko kya kehte hain?\n\nJawab: Cheesy!\n\nYeh joke hai ek shabd khel jismein "cheesy" ka matlab hai kuchh cheez jo bahut hi aam aur pasandida hoti hai, lekin yeh shabdon ka istemaal aise kar diya gaya hai jaise yeh ek jhootha shabd ho. Ismein zuban 

# Time Travel

In [ ]:
config2={'configurable':{'thread_id':thread_id_n}}
result=chatbot.invoke(
    {'topic':'burger'},
    config=config2
)

Generation completed...
Explanation completed...


In [ ]:
history=list(chatbot.get_state_history(config=config2))

In [ ]:
history

[StateSnapshot(values={'topic': 'burger', 'joke': AIMessage(content='Why did the burger go to therapy? \n\nBecause it was feeling a little crumby.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 46, 'total_tokens': 66, 'completion_time': 0.018759797, 'prompt_time': 0.10406644, 'queue_time': 0.06509964, 'total_time': 0.122826237}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_2115512ff6', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--844c251c-aedd-42fe-9c41-29da55b480f6-0', usage_metadata={'input_tokens': 46, 'output_tokens': 20, 'total_tokens': 66}), 'explain': AIMessage(content='Main ne ek burger ko pehli bar dekha tha aur usne mere se poocha, "Kya tum mujhe khatam kar sakte ho?"\n\nMain ne kehna shuru kiya, "Kuch hi karunga, main aapko khatam to kar sakta hoon, lekin aapko khaane mein behter lagta hai."', additional_kwargs={}, response_metadata={'token_usage': {'completion_token

In [ ]:
for i in history:
    print(i.config,i.values)

{'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f08b39d-92ef-6c7f-8006-fc5aeae87213'}} {'topic': 'burger', 'joke': AIMessage(content='Why did the burger go to therapy? \n\nBecause it was feeling a little crumby.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 46, 'total_tokens': 66, 'completion_time': 0.018759797, 'prompt_time': 0.10406644, 'queue_time': 0.06509964, 'total_time': 0.122826237}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_2115512ff6', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--844c251c-aedd-42fe-9c41-29da55b480f6-0', usage_metadata={'input_tokens': 46, 'output_tokens': 20, 'total_tokens': 66}), 'explain': AIMessage(content='Main ne ek burger ko pehli bar dekha tha aur usne mere se poocha, "Kya tum mujhe khatam kar sakte ho?"\n\nMain ne kehna shuru kiya, "Kuch hi karunga, main aapko khatam to kar sakta hoon, lekin aapko khaane mein be

In [ ]:
config2={'configurable':{'thread_id':thread_id_n,'checkpoint_id':'1f08b397-c032-6d17-8002-93219853b5ad'}}

In [ ]:

chatbot.invoke(None,config=config2)

{'topic': 'pizza',
 'joke': AIMessage(content='Why was the pizza in a bad mood? \n\nBecause it was feeling a little crusty.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 46, 'total_tokens': 66, 'completion_time': 0.016620453, 'prompt_time': 0.005892534, 'queue_time': 0.049852006, 'total_time': 0.022512987}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_7083106d2c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--13a56d55-6fb0-472e-b74e-5109d99afa62-0', usage_metadata={'input_tokens': 46, 'output_tokens': 20, 'total_tokens': 66}),
 'explain': AIMessage(content='Kuchh logon ko yeh joke pasand aati hai:\n\nPizza ki zuban ko kya kehte hain?\n\nJawab: Cheesy!\n\nYeh joke hai ek shabd khel jismein "cheesy" ka matlab hai kuchh cheez jo bahut hi aam aur pasandida hoti hai, lekin yeh shabdon ka istemaal aise kar diya gaya hai jaise yeh ek jhootha shabd ho. Ismein zuban ki cheez (pizza) ko 

# update state

In [75]:
chatbot.update_state(
    config={
        'configurable':{
            'thread_id':'1',
            'checkpoint_id':'1f08b397-c032-6d17-8002-93219853b5ad',
            'checkpoint_ns':''
            }
            },
                     values={'topic':'fries'}
                     )

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f08b607-050b-61ea-8000-c51873769598'}}

In [76]:
list(chatbot.get_state_history(config={'configurable':{'thread_id':'1'}}))

[StateSnapshot(values={'topic': 'fries'}, next=('joke_gen',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f08b607-050b-61ea-8000-c51873769598'}}, metadata={'source': 'update', 'step': 0, 'parents': {}}, created_at='2025-09-06T20:31:16.594322+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f08b397-c032-6d17-8002-93219853b5ad'}}, tasks=(PregelTask(id='37a89252-caa4-4d3c-f6ee-4d1f22a95ce8', name='joke_gen', path=('__pregel_pull', 'joke_gen'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': AIMessage(content='Why was the pizza in a bad mood? \n\nBecause it was feeling crusty.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 46, 'total_tokens': 64, 'completion_time': 0.016073858, 'prompt_time': 0.015830292, 'queue_time': 0.089567698, 'total_time': 0.03190415}, 'model_name': 'llama-

In [77]:
chatbot.invoke(
    None,config={
        'configurable':{'thread_id':'1','checkpoint_id':'1f08b607-050b-61ea-8000-c51873769598'}
    }
)

{'topic': 'fries',
 'joke': AIMessage(content='Why did the fries go to therapy? \n\nBecause they were struggling to cope with the heat of the moment.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 47, 'total_tokens': 70, 'completion_time': 0.042605748, 'prompt_time': 0.016848542, 'queue_time': 0.090819568, 'total_time': 0.05945429}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_7083106d2c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--4a8b8990-a5ae-413e-b3bd-592a615b9f6f-0', usage_metadata={'input_tokens': 47, 'output_tokens': 23, 'total_tokens': 70}),
 'explain': AIMessage(content='Main samajh nahi sakta ki kya joke diya gaya hai. Kya aap is joke ko explain karne ke liye ready hain?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 43, 'total_tokens': 79, 'completion_time': 0.068504242, 'prompt_time': 0.012168159, 'queue_time':

In [78]:
list(chatbot.get_state_history(config={'configurable':{'thread_id':'1'}}))

[StateSnapshot(values={'topic': 'fries', 'joke': AIMessage(content='Why did the fries go to therapy? \n\nBecause they were struggling to cope with the heat of the moment.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 47, 'total_tokens': 70, 'completion_time': 0.042605748, 'prompt_time': 0.016848542, 'queue_time': 0.090819568, 'total_time': 0.05945429}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_7083106d2c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--4a8b8990-a5ae-413e-b3bd-592a615b9f6f-0', usage_metadata={'input_tokens': 47, 'output_tokens': 23, 'total_tokens': 70}), 'explain': AIMessage(content='Main samajh nahi sakta ki kya joke diya gaya hai. Kya aap is joke ko explain karne ke liye ready hain?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 43, 'total_tokens': 79, 'completion_time': 0.068504242, 'prompt_time': 0.0121